In [12]:
%matplotlib inline
import pandas as pd
import os
import numpy as np
import seaborn as sns
import glob
import matplotlib.pyplot as plt
import sklearn


In [13]:
NEWLINE = '\n'
SKIP_FILES = {'cmds'}


def read_files(path):
    for root, dir_names, file_names in os.walk(path):
        for path in dir_names:
            read_files(os.path.join(root, path))
        for file_name in file_names:
            if file_name not in SKIP_FILES:
                file_path = os.path.join(root, file_name)
                if os.path.isfile(file_path):
                    past_header, lines = False, []
                    f = open(file_path, encoding="ANSI")
                    for line in f:
                        if past_header:
                            lines.append(line)
                        elif line == NEWLINE:
                            past_header = True
                    f.close()
                    content = NEWLINE.join(lines)
                    yield content
                    


In [14]:
path =r'C:\Users\osutr_000\Documents\Data\Ops'
list_ = []

for value in read_files(path):
    list_.append(value)

ops_df = pd.DataFrame(data = list_)
ops_df['class']="ops"
len(ops_df)

580

In [15]:
path =r'C:\Users\osutr_000\Documents\Data\Legal'
list_ = []

for value in read_files(path):
    list_.append(value)

legal_df = pd.DataFrame(data = list_)
legal_df['class']="legal"
len(legal_df)

209

In [16]:
path =r'C:\Users\osutr_000\Documents\Data\Accounting'
list_ = []

for value in read_files(path):
    list_.append(value)

act_df = pd.DataFrame(data = list_)
act_df['class']="accounting"
len(act_df)

324

In [17]:
merged_df = ops_df.append(legal_df).append(act_df)
len(merged_df)
merged_df.columns = ['text', 'cat']
merged_df.sort_index().reset_index().head()

,index,text,cat
0,0,"401 W. 33rd \n\nEdmond, OK 73013 \n\nWWW.MIDCO...",ops
1,0,26938 \n\n\n\n002 \n\n\n\n4/27/16 290.60 290. ...,accounting
2,0,"MIDCON Data Services, LLC. \n\n\n\nThis Data D...",legal
3,1,5/25/16 179.17 179.17 \n\n\n\n\n\n5/27/16 0160...,accounting
4,1,,legal


In [18]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from __future__ import print_function
from pprint import pprint
from time import time
import logging

In [19]:
# Flatten data into a (sample, feature) matrix
X = merged_df.text
y = merged_df.cat

# and then split the dataset into two parts
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.8, random_state=0)

In [20]:
# Define a pipeline combining a text feature extractor with a simple
# classifier
pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier()),
])

# uncommenting more parameters will give better exploring power but will
# increase processing time in a combinatorial way
parameters = {
    'vect__max_df': (0.5, 0.75, 1.0),
    #'vect__max_features': (None, 5000, 10000, 50000),
    'vect__ngram_range': ((1, 1), (1, 2)),  # unigrams or bigrams
    'tfidf__use_idf': (True, False),
    #'tfidf__norm': ('l1', 'l2'),
    #'clf__max_iter': (5,),
    'clf__alpha': (0.00001, 0.000001),
    'clf__penalty': ('l2', 'elasticnet'),
    #'clf__n_iter': (10, 50, 80),
}


In [23]:
scores = ['precision', 'recall']
for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(pipeline, parameters, cv=5,
                       scoring='%s_macro' % score)
    clf.fit(X_train, y_train)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, clf.predict(X_test)
    print(classification_report(y_true, y_pred))
    print()

# Tuning hyper-parameters for precision

Best parameters set found on development set:

{'clf__alpha': 1e-05, 'clf__penalty': 'l2', 'tfidf__use_idf': True, 'vect__max_df': 0.75, 'vect__ngram_range': (1, 2)}

Grid scores on development set:

0.971 (+/-0.072) for {'clf__alpha': 1e-05, 'clf__penalty': 'l2', 'tfidf__use_idf': True, 'vect__max_df': 0.5, 'vect__ngram_range': (1, 1)}
0.983 (+/-0.048) for {'clf__alpha': 1e-05, 'clf__penalty': 'l2', 'tfidf__use_idf': True, 'vect__max_df': 0.5, 'vect__ngram_range': (1, 2)}
0.965 (+/-0.066) for {'clf__alpha': 1e-05, 'clf__penalty': 'l2', 'tfidf__use_idf': True, 'vect__max_df': 0.75, 'vect__ngram_range': (1, 1)}
0.983 (+/-0.066) for {'clf__alpha': 1e-05, 'clf__penalty': 'l2', 'tfidf__use_idf': True, 'vect__max_df': 0.75, 'vect__ngram_range': (1, 2)}
0.971 (+/-0.072) for {'clf__alpha': 1e-05, 'clf__penalty': 'l2', 'tfidf__use_idf': True, 'vect__max_df': 1.0, 'vect__ngram_range': (1, 1)}
0.977 (+/-0.065) for {'clf__alpha': 1e-05, 'clf__penalty': 'l2

0.978 (+/-0.065) for {'clf__alpha': 1e-05, 'clf__penalty': 'elasticnet', 'tfidf__use_idf': True, 'vect__max_df': 0.5, 'vect__ngram_range': (1, 2)}
0.973 (+/-0.062) for {'clf__alpha': 1e-05, 'clf__penalty': 'elasticnet', 'tfidf__use_idf': True, 'vect__max_df': 0.75, 'vect__ngram_range': (1, 1)}
0.973 (+/-0.046) for {'clf__alpha': 1e-05, 'clf__penalty': 'elasticnet', 'tfidf__use_idf': True, 'vect__max_df': 0.75, 'vect__ngram_range': (1, 2)}
0.981 (+/-0.049) for {'clf__alpha': 1e-05, 'clf__penalty': 'elasticnet', 'tfidf__use_idf': True, 'vect__max_df': 1.0, 'vect__ngram_range': (1, 1)}
0.983 (+/-0.044) for {'clf__alpha': 1e-05, 'clf__penalty': 'elasticnet', 'tfidf__use_idf': True, 'vect__max_df': 1.0, 'vect__ngram_range': (1, 2)}
0.972 (+/-0.063) for {'clf__alpha': 1e-05, 'clf__penalty': 'elasticnet', 'tfidf__use_idf': False, 'vect__max_df': 0.5, 'vect__ngram_range': (1, 1)}
0.978 (+/-0.065) for {'clf__alpha': 1e-05, 'clf__penalty': 'elasticnet', 'tfidf__use_idf': False, 'vect__max_df': 0